In [5]:
import module.TorchDataset as td
import platform
import importlib

importlib.reload(td) # 클래스 수정 시 import 리로드

# os에 따라 다르게 적용 (코랩은 리눅스)
current_os = platform.system()
sep = "\\" if current_os == "Windows" else "/" 


# TrainSet
root_train = f"..{sep}01.Data{sep}fire{sep}train{sep}" # 루트폴더
extension = "jpg" # 찾을 확장자
img_size = 224

train_dataset = td.TorchDataset(root_train, sep, extension, img_size, "train", False)


# ValidSet
root_valid = f"..{sep}01.Data{sep}fire{sep}valid{sep}" # 루트폴더
extension = "jpg" # 찾을 확장자

valid_dataset = td.TorchDataset(root_valid, sep, extension, img_size, "test", False)


# TestSet
root_test = f"..{sep}01.Data{sep}fire{sep}test{sep}" # 루트폴더
extension = "jpg" # 찾을 확장자

test_dataset = td.TorchDataset(root_test, sep, extension, img_size, "test", False)

len(train_dataset), len(valid_dataset), len(test_dataset)

(30249, 6300, 6299)

In [ ]:
# print(train_dataset.get_label_dict())
# train_dataset.get_sample_image(8, True)
# valid_dataset.get_sample_image(8, True)
# test_dataset.get_sample_image(8, True)

In [ ]:
# train_dataset.get_target_ratio("Train")
# valid_dataset.get_target_ratio("Valid")
# test_dataset.get_target_ratio("Test")

In [6]:
import module.TorchDatasetCNN as tdCNN
importlib.reload(tdCNN)

cnn_runner = tdCNN.TorchDatasetCNN()
batch_size = 20

train_dataloader = cnn_runner.get_dataloader(train_dataset, batch_size, True, True)
valid_dataloader = cnn_runner.get_dataloader(valid_dataset, batch_size, False, False)
test_dataloader = cnn_runner.get_dataloader(test_dataset, batch_size, True, False)
len(train_dataloader), len(valid_dataloader), len(test_dataloader)

(1512, 315, 315)

In [ ]:
######################### ResNET50 ########################
###########################################################
###########################################################
###########################################################

In [14]:
import torch.nn as nn
from torchvision import models
from torchsummary import summary

epoch = 10
device = cnn_runner.get_device()
learning_rate = 0.001
save_path = ""

save_path = ""
if current_os == "Windows": # 로컬 저장용
    save_path = f"save_model{sep}03.Torch_CNN{sep}06.googlenet_Finetune{sep}googlenet_Finetune.pth"
else:
    save_path = "/content/googlenet_Finetune.pth" # 코랩 저장용

# model = models.googlnet50(weights='GooglNet50_Weights.DEFAULT').to(device)
model = models.googlenet(weights='DEFAULT').to(device)

# 모든 파라미터 Freezing
for param in model.parameters():
    param.requires_grad = False
    
# 마지막 레이어 layer4 파라미터만 풀어주기
for param in model.inception5b.parameters():
    param.requires_grad = True

# fc layer만 수정
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# trainable parameter 수 확인
summary(model, input_size = (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
       BasicConv2d-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
       BasicConv2d-7           [-1, 64, 56, 56]               0
            Conv2d-8          [-1, 192, 56, 56]         110,592
       BatchNorm2d-9          [-1, 192, 56, 56]             384
      BasicConv2d-10          [-1, 192, 56, 56]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,288
      BatchNorm2d-13           [-1, 64, 28, 28]             128
      BasicConv2d-14           [-1, 64,

In [15]:
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
# Train
train_acc, valid_acc, valid_recall = cnn_runner.run_epoch(epoch, train_dataloader, valid_dataloader, device, model, learning_rate, save_path)

In [ ]:
# 모델 로딩 및 테스트

save_path = ""
if current_os == "Windows": # 로컬 저장용
    save_path = f"save_model{sep}03.Torch_CNN{sep}06.googlnet_Finetune{sep}googlenet_Finetune.pth"
else:
    save_path = "/content/googlenet_Finetune.pth" # 코랩 저장용

model = cnn_runner.load_model(model, save_path, device)
# cnn_runner.test_model(test_dataloader, device, model)